In [1]:
import pandas as pd

In [179]:
df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/ML_job_postings_finale_ready_v2.csv',header=0)
df.head()

,job_id,title,work_type,description,location,zip_code,min_salary,max_salary,normalized_salary,views,...,sector,city,state,name,latitude,longitude,gdp,income,region,moving_score
0,1014822088,Marketing Specialist,FULL_TIME,About TrackerTracker is a leading global cloud...,United States,NaN,70000.0,80000.0,75000.0,7.0,...,Information Technology,United States,US,Remote,NaN,NaN,483162.0,416325.0,Plains,50.000000
1,3189117072,Client Service Associate / Practice Manager,FULL_TIME,Company OverviewSignature Estate and Investmen...,"Redondo Beach, CA",90277.0,70000.0,100000.0,85000.0,6.0,...,Energy,Redondo Beach,CA,California,36.778261,-119.417932,3870379.0,3166135.0,Far West,4.214518
2,3260455681,Operations Specialist,FULL_TIME,Weyerhaeuser is searching for a Operations Spe...,"Castle Rock, WA",98611.0,72742.0,109113.0,90927.5,18.0,...,Materials,Castle Rock,WA,Washington,47.751074,-120.740139,807865.0,632296.0,Far West,4.703287
3,3277232283,HR & Administrative Assistant,FULL_TIME,HR & Administrative CoordinatorWork is Remote ...,United States,NaN,55000.0,72000.0,63500.0,59.0,...,Energy,United States,US,Remote,NaN,NaN,483162.0,416325.0,Plains,50.000000
4,3297645846,Contract Administrator,FULL_TIME,Contract AdministratorSummary of PositionThe C...,"Broomfield, CO",80020.0,74400.0,90000.0,82200.0,6.0,...,Materials,Broomfield,CO,Colorado,39.550051,-105.782067,529627.0,470606.0,Rocky Mountain,8.368978


In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
df['economic_score'] = pca.fit_transform(df[['gdp', 'income']])

In [127]:
from sklearn.impute import SimpleImputer
# 1. Handle missing values for 'views' and 'company_size' using mean imputation
mean_imputer = SimpleImputer(strategy='mean')
df[['views', 'company_size']] = mean_imputer.fit_transform(df[['views', 'company_size']])

# 2. Handle missing values for 'formatted_experience_level' using model-based imputation
# For this, we need to build a predictive model for the missing values in 'formatted_experience_level'

# First, let's split the data into rows with missing and non-missing values
missing_experience_level = df[df['formatted_experience_level'].isnull()]
non_missing_experience_level = df[df['formatted_experience_level'].notnull()]

# Features used to predict 'formatted_experience_level'
features = ['min_salary', 'max_salary', 'normalized_salary', 'day_posting', 'programming_languages_score',
            'analysis_statistics_score', 'machine_learning_score', 'data_visualization_score', 
            'cloud_platform_score', 'database_score', 'big_data_tools_score', 'big_data_score', 
            'business_engineering_score', 'company_size', 'employee_count', 'follower_count', 'sector', 
            'state', 'latitude', 'longitude', 'region', 'moving_score']

# Ensure no missing values in features for model training
non_missing_experience_level = non_missing_experience_level.dropna(subset=features)

In [141]:
df=non_missing_experience_level

In [143]:
dt_df=df[['normalized_salary','views', 'day_posting', 'formatted_experience_level',
          'degree_level','job_classification', 'programming_languages_score',
          'analysis_statistics_score', 'machine_learning_score', 'data_visualization_score',
          'cloud_platform_score', 'database_score', 'big_data_tools_score', 'big_data_score',
          'business_engineering_score', 'company_size', 'employee_count', 'follower_count',
          'sector', 'state', 'latitude', 'longitude',  'region', 'moving_score'
            ]]

In [145]:
# Extract Features (X) and Target (y)
#define IVs and DV

X=dt_df.drop('normalized_salary',axis=1)
y=dt_df['normalized_salary']

In [147]:
X.isnull().sum()

views                          0
day_posting                    0
formatted_experience_level     0
degree_level                   0
job_classification             0
programming_languages_score    0
analysis_statistics_score      0
machine_learning_score         0
data_visualization_score       0
cloud_platform_score           0
database_score                 0
big_data_tools_score           0
big_data_score                 0
business_engineering_score     0
company_size                   0
employee_count                 0
follower_count                 0
sector                         0
state                          0
latitude                       0
longitude                      0
region                         0
moving_score                   0
dtype: int64

In [149]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['formatted_experience_level'] = label_encoder.fit_transform(X['formatted_experience_level'])

In [151]:
# One-Hot Encoding for Categorical Variables
X = pd.get_dummies(X, drop_first=True)  # drop_first to avoid multicollinearity

In [153]:
X.columns

Index(['views', 'day_posting', 'formatted_experience_level',
       'programming_languages_score', 'analysis_statistics_score',
       'machine_learning_score', 'data_visualization_score',
       'cloud_platform_score', 'database_score', 'big_data_tools_score',
       'big_data_score', 'business_engineering_score', 'company_size',
       'employee_count', 'follower_count', 'latitude', 'longitude',
       'moving_score', 'degree_level_bachelor, postgraduate',
       'degree_level_postgraduate', 'job_classification_Engineering/Software',
       'sector_Consumer Discretionary', 'sector_Consumer Staples',
       'sector_Energy', 'sector_Financials', 'sector_Health Care',
       'sector_Industrials', 'sector_Information Technology',
       'sector_Materials', 'sector_Real Estate', 'sector_Utilities',
       'state_AL', 'state_AR', 'state_AZ', 'state_CA', 'state_CO', 'state_CT',
       'state_DC', 'state_DE', 'state_FL', 'state_GA', 'state_HI', 'state_IA',
       'state_ID', 'state_IL', 'sta

In [155]:
# standardization
from sklearn import preprocessing
x_scaled=preprocessing.scale(X)

In [159]:
from sklearn.tree import DecisionTreeRegressor

# Split Data into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Predictions and performance evaluation
y_pred = dt_model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print(f"Decision Tree R²: {r2_score(y_test, y_pred):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}")

Decision Tree R²: -0.2012
Mean Absolute Error: 38617.4707
Mean Squared Error: 3024102167.3299


In [163]:
from sklearn.model_selection import GridSearchCV

# Define the Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42)

# Hyperparameter grid to tune
param_grid = {
    'max_depth': [3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

# Perform Grid Search for hyperparameter tuning
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_dt_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_dt_model.predict(X_test)
r2_score = best_dt_model.score(X_test, y_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Best Decision Tree Model Performance:")
print(f"R²: {r2_score:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/opt/anacond

Best Decision Tree Model Performance:
R²: 0.0982
Mean Absolute Error: 36411.5894
Mean Squared Error: 2270394586.7579


In [165]:
from sklearn.ensemble import RandomForestRegressor

# Define the Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform GridSearchCV to tune hyperparameters
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)

# Get the best Random Forest model
best_rf_model = grid_search_rf.best_estimator_

# Make predictions with the best model
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate the Random Forest model
r2_rf = best_rf_model.score(X_test, y_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)

print(f"Random Forest Model Performance:")
print(f"R²: {r2_rf:.4f}")
print(f"Mean Absolute Error: {mae_rf:.4f}")
print(f"Mean Squared Error: {mse_rf:.4f}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomFore

Random Forest Model Performance:
R²: 0.3432
Mean Absolute Error: 30545.1517
Mean Squared Error: 1653507376.9365


In [177]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
import numpy as np


# Initialize XGBoost Regressor
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = 5, alpha = 10, n_estimators = 10)

# Train the model
xg_reg.fit(X_train, y_train)

# Predictions on test set
y_pred = xg_reg.predict(X_test)

# Evaluate the model performance
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Print performance metrics
print(f"XGBoost Model Performance:")
print(f"R²: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Perform K-fold cross-validation to check model's stability
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(xg_reg, x_scaled, y, cv=kf, scoring='r2')

# Print K-Fold cross-validation results
print(f"K-Fold Cross-Validation (R²):")
print(f"Mean R²: {np.mean(cv_scores):.4f}, Std: {np.std(cv_scores):.4f}")

XGBoost Model Performance:
R²: 0.1318
Mean Absolute Error (MAE): 36389.5016
Mean Squared Error (MSE): 2185776669.1489
K-Fold Cross-Validation (R²):
Mean R²: 0.0986, Std: 0.0226
